<a href="https://colab.research.google.com/github/aspades1/PYOMO-PULP-Supply-chain-problem/blob/main/Supply_Chain_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
!pip install pulp
import pulp
import io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.2 MB 5.1 MB/s 


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Supply chain logisitcs problem.xlsx to Supply chain logisitcs problem (1).xlsx


In [ ]:
orders = pd.read_excel('Supply chain logisitcs problem.xlsx')
orders.dropna(axis = 1, how = 'all', inplace = True)
orders.dropna(axis = 0, how = 'all', inplace = True)
#orders.head()
#Total = orders['Unit quantity'].sum() #the sum of quantity will not matter in as a constraint, since plant capacity is in number of orders
#print(Total)

In [ ]:
#a=orders.groupby(['Order ID','Product ID']).size().reset_index().rename(columns={0:'count'})
#a[a['count']>1]
a=orders.groupby(['Order ID'])['Product ID'].count().reset_index()
a[a['Product ID']>1]

,Order ID,Product ID


In [ ]:
freight_rates = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name  = 1)
freight_rates.dropna(axis = 1, how = 'all', inplace = True)
freight_rates.dropna(axis = 0, how = 'all', inplace = True)
#freight_rates.head()

In [ ]:
wh_capacities = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name  = 2)
wh_capacities.dropna(axis = 1, how = 'all', inplace = True)
wh_capacities.dropna(axis = 0, how = 'all', inplace = True)
#wh_capacities.head()
#Total = wh_capacities['Daily Capacity'].sum()
#print(Total)

In [ ]:
products_per_plant = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name  = 3)
products_per_plant.dropna(axis = 1, how = 'all', inplace = True)
products_per_plant.dropna(axis = 0, how = 'all', inplace = True)
#products_per_plant.head()
#a=products_per_plant.groupby(['Product ID'])['Plant Code'].count().reset_index()
#a[a['Plant Code']>3]
#products_per_plant['Plant Code'].nunique() #number of unique factories

In [ ]:
ports = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name  = 4)
ports.dropna(axis = 1, how = 'all', inplace = True)
ports.dropna(axis = 0, how = 'all', inplace = True)
#ports.head()

In [ ]:
#a=ports.groupby(['Plant Code','Port']).size().reset_index().rename(columns={0:'count'}) #to verify more than one port per plant:
#a[a['count']>0]

#a=ports.groupby(['Plant Code'])['Port'].count().reset_index()
#a[a['Port']>1]

In [ ]:
shipping_costs = dict(zip(freight_rates['orig_port_cd'], freight_rates['rate']))

In [ ]:
def get_plants(product_id):
    
    temp = products_per_plant[products_per_plant['Product ID'] == product_id]
    return tuple(np.unique(temp['Plant Code']))

#for i,s in orders.iterrows():    #to verify more than one plant per id:
#   if len(orders['allowed_plants'][i])>1:
#       print(orders['Product ID'][i])
orders['allowed_plants']=orders['Product ID'].apply(get_plants)

In [ ]:
orders.set_index('Order ID',inplace=True)

In [ ]:
plant_ports = dict(zip(ports['Plant Code'], ports['Port']))

In [ ]:
#a=orders.groupby(['Order ID','Product ID']).size().reset_index().rename(columns={0:'count'})
#a[a['count']>1] #each order = unique product:

def production_cost(order_id, plant):

    temp_orders=orders.loc[order_id]
    temp_plants_costs = products_per_plant[products_per_plant['Product ID'] == temp_orders['Product ID']]
    temp_plants_costs = temp_plants_costs[temp_plants_costs['Plant Code'] == plant]['Cost per unit']
    production_cost = temp_plants_costs * temp_orders['Unit quantity']
    return production_cost.iloc[0]

In [ ]:
def shipping_cost(order_id, plant):

    temp_orders=orders.loc[order_id]
    temp_weight = temp_orders['Weight']
    
    temp_port = plant_ports[plant]
    temp_shippingc= shipping_costs[temp_port]
    
    cost = temp_shippingc * temp_weight
    return cost

In [ ]:
def total_cost(order_id, plant):

    a=shipping_cost(order_id, plant) + production_cost(order_id, plant)
    
    return a

In [ ]:
order_costs = {}
for name, row in orders.iterrows():
    order_id = name
    for plant in row['allowed_plants']:   
        order_costs[str(order_id) + '_' + str(plant)] = total_cost(order_id, plant)

### We create a dictionary with the key-value pair 'plantName': list_of_orders.

plants = np.unique(ports['Plant Code'])

plant_orders = {}
for plant in plants:
    temp_list = []
    for name, row in orders.iterrows():
        if plant in row['allowed_plants']:  
            temp_list.append(str(name) + '_' + plant)
    plant_orders[plant] = temp_list

### We create a dictionary with the key-value pair 'plantName': capacity.

plant_cap = dict(zip(wh_capacities['Plant ID'], wh_capacities['Daily Capacity']))

### We create a dictionary with the key-value pair 'orderID': orderID_plantName.
order_plants = {}
temp_dict = dict(zip(orders.index, orders['allowed_plants']))
for key in temp_dict:
    temp_list = []
    for pl in temp_dict[key]:
        temp_list.append(str(key) + '_' + pl)
    order_plants[key] = temp_list

#'''Preview of dictionaries:'''
#order_plants[783171]=['783171_PLANT03', '783171_PLANT09']#

#order_costs['783171_PLANT03']=4352
#order_costs['783171_PLANT09']=1075

#plant_orders['PLANT03']=[....., '783171_PLANT03',....]

In [ ]:
build = pulp.LpVariable.dicts("Route",order_costs.keys(),0,None, pulp.LpInteger)
prob = pulp.LpProblem("Problem",pulp.LpMinimize)
prob += pulp.lpSum([build[b] * order_costs[b] for b in order_costs.keys()]), "Total Cost"

In [ ]:
for plant in plant_orders:
    if len(plant_orders[plant]) > 0:
        prob += pulp.lpSum(build[p] for p in plant_orders[plant]) <= plant_cap[plant], "Total orders out of plant %s"%plant

for o in order_plants:
    prob += pulp.lpSum(build[p] for p in order_plants[o]) == 1, "Order_" + str(o) + "_filled"

In [ ]:
#Solving
status = pulp.LpStatus[prob.status]
for i in range(3):
    if status == 'Optimal':
        break
    prob.solve()
print("Status:", status)

Status: Optimal


In [ ]:
# Total cost
total_cost = round(pulp.value(prob.objective), 2)

print("Total Cost = ", str(total_cost))

Total Cost =  27140675.14


PYOMO

In [ ]:
!pip install pyomo
!apt-get install -y -qq glpk-utils
from pyomo.core.base.set_types import NonNegativeReals
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import pandas as pd
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Supply chain logisitcs problem.xlsx to Supply chain logisitcs problem.xlsx


In [ ]:
orders = pd.read_excel('Supply chain logisitcs problem.xlsx')
orders.dropna(axis = 1, how = 'all', inplace = True)
orders.dropna(axis = 0, how = 'all', inplace = True)

freight_rates = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name  = 1)
freight_rates.dropna(axis = 1, how = 'all', inplace = True)
freight_rates.dropna(axis = 0, how = 'all', inplace = True)

wh_capacities = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name  = 2)
wh_capacities.dropna(axis = 1, how = 'all', inplace = True)
wh_capacities.dropna(axis = 0, how = 'all', inplace = True)

products_per_plant = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name  = 3)
products_per_plant.dropna(axis = 1, how = 'all', inplace = True)
products_per_plant.dropna(axis = 0, how = 'all', inplace = True)

ports = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name  = 4)
ports.dropna(axis = 1, how = 'all', inplace = True)
ports.dropna(axis = 0, how = 'all', inplace = True)

In [ ]:
orders.head()
#a=orders.groupby(['Order ID'])['Product ID'].count().reset_index()
#a[a['Product ID']>1]
#''' Since each order has single product, it is easy to used product id to connect to plants'''

,Order ID,Order Date,Product ID,Destination Port,Unit quantity,Weight
0,782912,2019-12-31,1700106,PORT09,808,14.30
1,782913,2019-12-31,1700106,PORT09,3188,87.94
2,782914,2019-12-31,1700106,PORT09,2331,61.20
3,782915,2019-12-31,1700106,PORT09,847,16.16
4,782916,2019-12-31,1700106,PORT09,2163,52.34


In [ ]:
freight_rates.head() #'''dest_port_cd is always PORT09 so it is easy to exlude that and only concern ourselfs with origin'''

,orig_port_cd,dest_port_cd,rate
0,PORT02,PORT09,0.0484
1,PORT03,PORT09,0.1156
2,PORT04,PORT09,0.0620
3,PORT05,PORT09,0.1008
4,PORT06,PORT09,0.0848


In [ ]:
wh_capacities.head() #''' capacity in plant warehouse is not product_id specific'''

,Plant ID,Daily Capacity
0,PLANT15,1872
1,PLANT17,731
2,PLANT18,682
3,PLANT05,531
4,PLANT02,435


In [ ]:
products_per_plant.head()
#a=products_per_plant.groupby(['Product ID'])['Plant Code'].count().reset_index()
#a[a['Plant Code']>2]
#''' a single product can be produced in some occasions in multiple plants'''

,Plant Code,Product ID,Cost per unit
0,PLANT15,1698815,1.70
1,PLANT17,1664419,0.44
2,PLANT17,1664426,0.37
3,PLANT17,1672826,1.90
4,PLANT17,1674916,0.28


In [ ]:
ports.head()
#a=ports.groupby(['Plant Code'])['Port'].count().reset_index()
#a[a['Port']>1]
#'''in few occasions some plants have multiple ports (plant01,plant07,plant10) where they can be sent throught. That is easilt remedied as we can explude the ports with higher cost per weight'''
#ports[ports['Plant Code'].isin(['PLANT07', 'PLANT10', 'PLANT01'])]
#'''The respective ports are PORT01,PORT02'''

,Plant Code,Port
0,PLANT01,PORT01
1,PLANT01,PORT02
2,PLANT02,PORT03
3,PLANT03,PORT04
4,PLANT04,PORT05


In [ ]:
pc=ports.merge(freight_rates,how='left',left_on='Port', right_on='orig_port_cd')
pc[pc.isna().any(axis=1)]

freight_rates[freight_rates['orig_port_cd']=='PORT01']
#''' it turns that the port PORT01 was did not have available cost and thus is excluded'''
ports=ports[ports.Port!='PORT01']
pc=pc[pc.Port!='PORT01']
pc=pc[['Plant Code','Port','rate']]
#'''this affect the 'Port1'-'PLANT10',  'Port1'- 'PLANT07','Port1'-'PLANT01' connections, but since these plants also forward through PORT02 it is not a problem'''
#a=pc.groupby(['Plant Code'])['Port'].count().reset_index()
#a[a['Port']>0]
#'''The relationship plant-port is 1-1 now and can directly used to apply the costs'''

In [ ]:
#shipping_costs = dict(zip(freight_rates['orig_port_cd'], freight_rates['rate']))
def get_plants(product_id):

    temp=products_per_plant[products_per_plant['Product ID']==product_id]
    return tuple(np.unique(temp['Plant Code']))
orders['allowed_plants']=orders['Product ID'].apply(get_plants)

orders=orders.merge(products_per_plant,how='left',on='Product ID')
#to verify more than one plant per id:
#for i,s in orders.iterrows():
#    if len(orders['allowed_plants'][i])>1:
#        print(orders['Product ID'][i])

#orders[orders['Product ID']==1690616]

In [ ]:
orders=orders.merge(pc,how='left',on='Plant Code')

In [ ]:
orders['trc']=orders['rate']*orders['Weight']
orders['trc_perunit']=orders['trc']/orders['Unit quantity']
orders['pc']=orders['Cost per unit']*orders['Unit quantity']
orders['tc']=orders['trc']+orders['pc']
orders['tc_perunit']=orders['tc']/orders['Unit quantity']

In [ ]:
orders.head()

,Order ID,Order Date,Product ID,Destination Port,Unit quantity,Weight,allowed_plants,Plant Code,Cost per unit,Port,rate,trc,trc_perunit,pc,tc,tc_perunit
0,782912,2019-12-31,1700106,PORT09,808,14.30,"(PLANT16,)",PLANT16,0.46,PORT09,0.0332,0.474760,0.000588,371.68,372.154760,0.460588
1,782913,2019-12-31,1700106,PORT09,3188,87.94,"(PLANT16,)",PLANT16,0.46,PORT09,0.0332,2.919608,0.000916,1466.48,1469.399608,0.460916
2,782914,2019-12-31,1700106,PORT09,2331,61.20,"(PLANT16,)",PLANT16,0.46,PORT09,0.0332,2.031840,0.000872,1072.26,1074.291840,0.460872
3,782915,2019-12-31,1700106,PORT09,847,16.16,"(PLANT16,)",PLANT16,0.46,PORT09,0.0332,0.536512,0.000633,389.62,390.156512,0.460633
4,782916,2019-12-31,1700106,PORT09,2163,52.34,"(PLANT16,)",PLANT16,0.46,PORT09,0.0332,1.737688,0.000803,994.98,996.717688,0.460803


In [ ]:
order_perplant=orders.groupby('Plant Code')['Order ID'].nunique().to_dict()
'''we can see that in some cases there is an larger number of orders that can be satisfied from a factory, that the factory's capacity'''

"we can see that in some cases there is an larger number of orders that can be satisfied from a factory, that the factory's capacity"

In [ ]:
total_costs_per_unit = {}
for name, row in orders.iterrows():
  total_costs_per_unit [tuple([orders.iloc[name]['Order ID'],orders.iloc[name]['Plant Code']])]= orders.iloc[name]['tc_perunit']

total_costs= {}
for name, row in orders.iterrows():
  total_costs [tuple([orders.iloc[name]['Order ID'],orders.iloc[name]['Plant Code']])]= orders.iloc[name]['tc']

demand= dict(zip(orders['Order ID'], orders['Unit quantity']))

supply= dict(zip(wh_capacities['Plant ID'], wh_capacities['Daily Capacity']))

#Plants 17,15,19 do not produce products that are in our demand and will create error in constraint. We could either pop the plants:
supply.pop('PLANT17')
supply.pop("PLANT15")
supply.pop("PLANT19")

#supply = {k: supply[k] for k in orders['Plant Code'].unique().tolist()} #Or we can exlude them , knowing the true combinations

450

In [ ]:
'''
We will solve the proble in 2 different ways. 
1) As first intended, the constrains in production in number of orders not in quantities produced
->#Daily Capacity: Max number of orders processed per day. Note that this capacitycorresponds to number of orders, not units.
'''
#Concrete model
model=pyo.ConcreteModel()
#Sets
model.i=pyo.Set(initialize=demand.keys()) #orders
model.j=pyo.Set(initialize=supply.keys()) #factories
model.select_combos = pyo.Set(within = model.i * model.j, initialize = total_costs.keys())

#Parameters
model.p=pyo.Param(model.select_combos, initialize=total_costs) # here goes the cost dictionary

model.d=pyo.Param(model.i,initialize=demand)

model.s=pyo.Param(model.j,initialize=supply)

#Decision variable
model.x=pyo.Var(model.select_combos, within=pyo.NonNegativeReals)

#Objective function

#plant_used = pyo.summation(model.plants_binary, m.Assign)
model.Obj=pyo.Objective(expr=sum(model.p[i,j]*model.x[i,j] for i,j in model.select_combos),sense=pyo.minimize)

#Constraints
def Const1(model,i):
  return sum(model.x[i,j] for j in model.j if (i,j) in model.select_combos)>=1
model.condemand=pyo.Constraint(model.i,rule=Const1)

def Const6(model,j):
    return sum(model.x[i,j] for i in model.i if (i,j) in model.select_combos) <= model.s[j]
model.consupply=pyo.Constraint(model.j,rule=Const6)

#Solving
Solver=pyo.SolverFactory('glpk')

results=Solver.solve(model)

''' If we wanted to print information
#print('Obj funct=',model.Obj())
#for i, j in model.select_combos:  # if the we wanted to print them
  #print('for order',i," from plant ",j, " is sent ", pyo.value(model.x[i,j]))
#ind = list(model.select_combos). #if we want to save combination defined in model in a list
'''
print(results)

print('Obj funct=',model.Obj())

#Exporting results and convrting to Pandas Dataframe
ind = list(model.x)
val = list(model.x[:,:].value)
result_P = [ i + tuple([j]) for i,j in zip(ind, val)]
result_P = np.asarray(result_P)
df = pd.DataFrame(result_P,columns=['Order ID', 'Plant Code','Result'])
df['Order ID'] = df['Order ID'].astype(int)
res=orders.merge(df,how='inner', left_on=['Order ID', 'Plant Code'], right_on=['Order ID', 'Plant Code'])



Problem: 
- Name: unknown
  Lower bound: 27140675.1374655
  Upper bound: 27140675.1374655
  Number of objectives: 1
  Number of constraints: 9232
  Number of variables: 16246
  Number of nonzeros: 32491
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 2.3884358406066895
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Obj funct= 27140675.137465402


''' 2) If we were given constrait of production, in number of units produced instead the number of orders the it shoud be as follows:'''

In [ ]:
supply.update((x, y*5000) for x, y in supply.items()) #as we had limitation in number of orders instead of quantities of production, we are multiplying it to make the constraint saffisiently big to solve

In [ ]:
#Concrete model
model=pyo.ConcreteModel()

#Sets
model.i=pyo.Set(initialize=demand.keys()) #orders
model.j=pyo.Set(initialize=supply.keys()) #factories
model.select_combos = pyo.Set(within = model.i * model.j, initialize = total_costs_per_unit.keys())

#Parameters
model.p=pyo.Param(model.select_combos, initialize=total_costs_per_unit) # here goes the cost dictionary

model.d=pyo.Param(model.i,initialize=demand)

model.s=pyo.Param(model.j,initialize=supply)

#Decision variable
model.x=pyo.Var(model.select_combos, within=pyo.NonNegativeReals)

#Objective function

model.Obj=pyo.Objective(expr=sum(model.p[i,j]*model.x[i,j] for i,j in model.select_combos),sense=pyo.minimize)

#Constraints

def Const1(model,i):
  return sum(model.x[i,j] for j in model.j if (i,j) in model.select_combos)>=model.d[i]
model.condemand=pyo.Constraint(model.i,rule=Const1)

def Const5(model,j):
  return sum(model.x[i,j] for i in model.i if (i,j) in model.select_combos)<=model.s[j]
model.consupply=pyo.Constraint(model.j,rule=Const5)

#Solving
Solver=pyo.SolverFactory('glpk')

results=Solver.solve(model)

print(results)

print('Obj funct=',model.Obj())
#for i, j in model.select_combos:
#  print('for order',i," from plant ",j, " is sent ", pyo.value(model.x[i,j]))




Problem: 
- Name: unknown
  Lower bound: 27138772.9939761
  Upper bound: 27138772.9939761
  Number of objectives: 1
  Number of constraints: 9232
  Number of variables: 16246
  Number of nonzeros: 32491
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 1.8121435642242432
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Obj funct= 27138772.993976105
